<a href="https://colab.research.google.com/github/future-158/vision-labeling-humanintheloop/blob/master/eda_step.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive', force_remount=True)

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tqdm
from pathlib import Path
import cv2

In [ ]:
# train_dir = Path('/content/drive/MyDrive/project/buzzni/dataset/data/train')
# test_dir = Path('/content/drive/MyDrive/project/buzzni/dataset/data/test')
train_dir = Path('./dataset/data/train')
test_dir = Path('./dataset/data/test')

train_files = list(train_dir.glob('**/*.jpg'))
test_files = list(test_dir.glob('**/*.jpg'))
files = [*train_files, *test_files]
img_shapes = [cv2.imread(file.as_posix()).shape for file in tqdm.tqdm(files)]
widths = [x[0] for x in img_shapes]
heights = [x[1] for x in img_shapes]
resolutions = np.sqrt(np.array(widths) * np.array(heights)).astype(int)

table = pd.DataFrame(data=dict(file=files, img_shape = img_shapes))
table['split'] = table.file.astype(str).str.split('/').str.get(-3)
table['category'] = table.file.astype(str).str.split('/').str.get(-2)
table['resolution'] = resolutions
# widths, heights = list(zip(*[imagesize.get(file) for file in tqdm.tqdm(files)]))

# find inter-class distribution

In [ ]:
table.groupby('category')['file'].count().mean()

In [ ]:
(
    table
 .groupby(['category','split'])['file']
 .count()
 .reset_index()
 .pipe(lambda df: df.pivot_table(index='category',columns='split'))
 .droplevel(axis=1, level=0)
 .assign(test_ratio = lambda df: df['test'] / df['train'])
 ['test_ratio']
 .describe()
)

In [ ]:
table.groupby(['category'])['resolution'].mean().describe()

In [ ]:
(
table
 .groupby(['category','split'])['resolution']
 .mean()
 .reset_index()
 .pipe(lambda df: df.pivot_table(index='category',columns='split'))
 .droplevel(axis=1, level=0)
 .assign(enlarge_ratio = lambda df: df['test'] / df['train'])
#  ['enlarge_ratio'].mean()
 .sort_values(['enlarge_ratio'])
)